In [ ]:
import pandas as pd

In [ ]:
str_cols = ["SCHEDULED_DEPARTURE", "DEPARTURE_TIME","WHEELS_OFF","WHEELS_ON","SCHEDULED_ARRIVAL","ARRIVAL_TIME"]
df_flights=pd.read_csv("data/flights.csv",dtype={i:str for i in str_cols})
df_airlines=pd.read_csv("data/airlines.csv")
df_airports=pd.read_csv("data/airports.csv")

### Time your work, 2 hours max

### Each row is <AIRLINE> <AIRPORT>, each row is hour range (0-3am, 3-6am, ..., 9pm-12) total 8 session per day (edited) 

### table 1, total count of flight 

### table 2, average delay ratio excluding cancelled flight 

### table 3, average delay in minutes 

### delay definition: departure delay for operated flight 

### operated = non cancelled

### table 1

### df_flights: groupby AIRLINE, ORIGIN_AIRPORT, DEPARTURE_TIME, NEW COLUMN - DEPARTURE_TIME_HOUR

In [ ]:
# find operated flights
df_result=df_flights.loc[df_flights["CANCELLED"]!=1]
df_result["DEPARTURE_TIME_HOUR"]=[i[:2] for i in df_result["DEPARTURE_TIME"]]
data1=["0-2","0-2","0-2","3-5","3-5","3-5","6-8","6-8","6-8","9-11","9-11","9-11","12-14","12-14","12-14","15-17","15-17","15-17","18-20","18-20","18-20","21-23","21-23","21-23"]
rows1=["00","01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23"]
df_lookup=pd.DataFrame(data=rows1,columns=["HOUR"])
df_lookup["HOUR_RANGE"]=data1
df_result2=df_result.merge(df_lookup,how="left",left_on="DEPARTURE_TIME_HOUR",right_on="HOUR")
table1=pd.pivot_table(df_result2,values=["FLIGHT_NUMBER"],index=["AIRLINE","ORIGIN_AIRPORT"],columns=["HOUR_RANGE"],aggfunc=[len])
a=table1["len"]["FLIGHT_NUMBER"]
index1=["0-2","3-5","6-8","9-11","12-14","15-17","18-20","21-23"]
b=a.reindex(columns=index1)
b.to_excel("table1.xlsx")

### Table2

In [ ]:
# find operated flights
df_result=df_flights.loc[df_flights["CANCELLED"]!=1]
df_result["DEPARTURE_TIME_HOUR"]=[i[:2] for i in df_result["DEPARTURE_TIME"]]
data1=["0-2","0-2","0-2","3-5","3-5","3-5","6-8","6-8","6-8","9-11","9-11","9-11","12-14","12-14","12-14","15-17","15-17","15-17","18-20","18-20","18-20","21-23","21-23","21-23"]
rows1=["00","01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23"]
df_lookup=pd.DataFrame(data=rows1,columns=["HOUR"])
df_lookup["HOUR_RANGE"]=data1
df_result2=df_result.merge(df_lookup,how="left",left_on="DEPARTURE_TIME_HOUR",right_on="HOUR")


df_result2["isdelayed"]=df_result2["DEPARTURE_DELAY"]>0
df_result3=df_result2.groupby(by=["AIRLINE","HOUR_RANGE","ORIGIN_AIRPORT"]).agg({"AIRLINE":"count","isdelayed":"sum"})
df_result3["delay_ratio"]=df_result3["isdelayed"]/df_result3["AIRLINE"]
a=df_result3["delay_ratio"]
b=a.reset_index()
c=pd.pivot_table(b,values="delay_ratio",index=["AIRLINE","ORIGIN_AIRPORT"],columns="HOUR_RANGE",aggfunc="mean")
c.reindex(columns=["0-2","3-5","6-8","9-11","12-14","15-17","18-20","21-23"]).to_excel("table2.xlsx")

### Table3

In [ ]:
df_result3=df_flights.loc[df_flights["DEPARTURE_DELAY"]>0]
df_result3["DEPARTURE_TIME_HOUR"]=[i[:2] for i in df_result3["DEPARTURE_TIME"]]
data1=["0-2","0-2","0-2","3-5","3-5","3-5","6-8","6-8","6-8","9-11","9-11","9-11","12-14","12-14","12-14","15-17","15-17","15-17","18-20","18-20","18-20","21-23","21-23","21-23"]
rows1=["00","01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23"]
df_lookup=pd.DataFrame(data=rows1,columns=["HOUR"])
df_lookup["HOUR_RANGE"]=data1
df_result4=df_result3.merge(df_lookup,how="left",left_on="DEPARTURE_TIME_HOUR",right_on="HOUR")
df_result5=df_result4[["AIRLINE","ORIGIN_AIRPORT","DEPARTURE_DELAY","HOUR_RANGE"]]
df_result6=df_result5.pivot_table(values="DEPARTURE_DELAY",index=["AIRLINE","ORIGIN_AIRPORT"],columns="HOUR_RANGE",aggfunc="mean")
df_result6.reindex(columns=["0-2","3-5","6-8","9-11","12-14","15-17","18-20","21-23"]).round(1).to_excel("table3.xlsx")